Запуск сервера:
```bash
uv run --active uvicorn server.main:app --host 0.0.0.0 --port 8000
```

Проверка статуса сервера:
```bash
curl http://localhost:8000/status
```

# Демонстрация работы с ML-сервером

Посмотрим на работу клиентской части для взаимодействия с ML-сервером. Не забудьте установить все необходимые зависимости проекта.
Демонстрируются следующие функции:
- Синхронное обучение двух моделей (длительность ≥60 секунд для каждой).
- Асинхронное обучение двух моделей с ускорением.
- Асинхронный вызов нескольких предсказаний.
- Загрузка, выгрузка, удаление моделей и удаление всех моделей.

Есть обработка ошибок сервера (404, 429, 500) и сетевых исключений.

In [11]:
import requests
import aiohttp
import asyncio
import numpy as np
from sklearn.datasets import make_classification
import time
import json

# Базовый URL сервера
BASE_URL = "http://localhost:8000"

# Генерация синтетического датасета для обучения (длительность ≥60 секунд)
X, y = make_classification(
    n_samples=20000,  # Большое число объектов
    n_features=200,   # Большое число признаков
    n_classes=3,
    n_informative=5,
    random_state=42
)
X = X.tolist()
y = y.tolist()

# Данные для предсказания
X_pred = X[:100]  # Первые 10 объектов

# Конфигурации моделей
models = [
    {"name": "logreg_model", "type": "logistic_regression", "params": {}},
    {"name": "gb_model", "type": "gradient_boosting", "params": {"n_estimators": 2000}}
]

## 1. Синхронное обучение двух моделей

**Ожидаемое действие**: Обучаем две модели (`logistic_regression` и `gradient_boosting`) последовательно с помощью `requests`. Для `gradient_boosting` задаём `n_estimators=1000`, чтобы обучение длилось ≥60 секунд. Измеряем общее время. Обрабатываем возможные ошибки (например, отсутствие свободных ядер или превышение лимита моделей).

In [12]:
def train_sync(model_name: str, model_type: str, model_params: dict):
    """Синхронный вызов обучения модели."""
    payload = {
        "X": X,
        "y": y,
        "config": {
            "model_name": model_name,
            "model_type": model_type,
            "model_params": model_params
        }
    }
    try:
        start_time = time.time()
        response = requests.post(f"{BASE_URL}/fit", json=payload, timeout=300)
        response.raise_for_status()
        print(f"{model_name}: {response.json()['message']} (took {time.time() - start_time:.2f}s)")
    except requests.exceptions.HTTPError as e:
        if response.status_code == 429:
            print(f"Error for {model_name}: No available cores or max models reached: {response.json()['detail']}")
        elif response.status_code == 500:
            print(f"Error for {model_name}: Server error: {response.json()['detail']}")
        else:
            print(f"Error for {model_name}: {str(e)}")
    except requests.exceptions.RequestException as e:
        print(f"Network error for {model_name}: {str(e)}")

# Синхронное обучение
print("Starting synchronous training...")
sync_start = time.time()
for model in models:
    train_sync(model["name"], model["type"], model["params"])
sync_duration = time.time() - sync_start
print(f"Total synchronous training time: {sync_duration:.2f}s")

Starting synchronous training...
logreg_model: Training started for model_name 'logreg_model' (took 5.82s)
gb_model: Training started for model_name 'gb_model' (took 6.22s)
Total synchronous training time: 12.07s


## 2. Асинхронное обучение двух моделей

Теперь обучаем те же две модели **асинхронно**. Видим, что обучение выполняется быстрее, чем синхронное.

In [ ]:
async def train_async(session: aiohttp.ClientSession, model_name: str, model_type: str, model_params: dict):
    """Асинхронный вызов обучения модели."""
    payload = {
        "X": X,
        "y": y,
        "config": {
            "model_name": model_name,
            "model_type": model_type,
            "model_params": model_params
        }
    }
    try:
        start_time = time.time()
        async with session.post(f"{BASE_URL}/fit", json=payload, timeout=300) as response:
            data = await response.json()
        
            if response.status == 200:
                print(f"{model_name}: Training started")
                
                # Периодически проверяем статус
                while True:
                    await asyncio.sleep(5)
                    async with session.get(f"{BASE_URL}/status") as status_resp:
                        status = await status_resp.json()
                        if model_name in status['available_models']:
                            duration = time.time() - start_time
                            print(f"{model_name}: Training completed (took {duration:.2f}s)")
                            break
            elif response.status == 429:
                print(f"Error for {model_name}: No available cores or max models reached: {await response.json()['detail']}")
            elif response.status == 500:
                print(f"Error for {model_name}: Server error: {await response.json()['detail']}")
            else:
                print(f"Error for {model_name}: HTTP {response.status}")
    except aiohttp.ClientError as e:
        print(f"Network error for {model_name}: {str(e)}")

async def async_training():
    async with aiohttp.ClientSession() as session:
        tasks = [
            train_async(session, model["name"], model["type"], model["params"])
            for model in models
        ]
        await asyncio.gather(*tasks)

# Асинхронное обучение
print("Starting asynchronous training...")
async_start = time.time()
await async_training()
async_duration = time.time() - async_start
print(f"Total asynchronous training time: {async_duration:.2f}s")
print(f"Speedup: {sync_duration / async_duration:.2f}x")

Starting asynchronous training...
gb_model: Training started for model_name 'gb_model' (took 4.19s)
logreg_model: Training started for model_name 'logreg_model' (took 7.96s)
Total asynchronous training time: 7.97s
Speedup: 1.09x


In [ ]:
async def wait_for_training_completion(session: aiohttp.ClientSession, model_name: str, timeout: int = 300):
    """Ожидает завершения обучения модели"""
    start_time = time.time()
    while time.time() - start_time < timeout:
        # Проверяем статус сервера
        async with session.get(f"{BASE_URL}/status") as resp:
            status = await resp.json()
            available_models = list(Path(MODEL_DIR).glob("*.pkl"))
            model_files = [m.stem for m in available_models]
            
            if model_name in model_files:
                return True
            
        await asyncio.sleep(5)
    
    raise TimeoutError(f"Training for {model_name} didn't complete in {timeout} seconds")

async def load_model(session: aiohttp.ClientSession, model_name: str):
    """Загружает модель в память сервера"""
    payload = {"config": {"model_name": model_name}}
    async with session.post(f"{BASE_URL}/load", json=payload) as resp:
        if resp.status == 200:
            return True
        error = await resp.json()
        raise ValueError(f"Failed to load {model_name}: {error.get('detail')}")

async def async_training_and_predict():
    async with aiohttp.ClientSession() as session:
        # Запускаем обучение
        train_tasks = [
            train_async(session, model["name"], model["type"], model["params"])
            for model in models
        ]
        await asyncio.gather(*train_tasks)
        
        # Ожидаем завершения обучения
        print("Waiting for training to complete...")
        await asyncio.gather(*[
            wait_for_training_completion(session, model["name"])
            for model in models
        ])
        
        # Загружаем модели
        print("Loading models...")
        await asyncio.gather(*[
            load_model(session, model["name"])
            for model in models
        ])
        
        # Теперь делаем предсказания
        print("Starting predictions...")
        await async_predictions()

# Запуск всего pipeline
print("Starting full training and prediction pipeline...")
start_time = time.time()
await async_training_and_predict()
print(f"Total time: {time.time() - start_time:.2f}s")

## 3. Асинхронный вызов нескольких предсказаний

Сделаем асинхронные предсказания для двух моделей одновременно. Используем небольшой набор данных (`X_pred`).

In [10]:
async def predict_async(session: aiohttp.ClientSession, model_name: str):
    """Асинхронный вызов предсказания."""
    payload = {
        "y": X_pred,
        "config": {"model_name": model_name}
    }
    try:
        async with session.post(f"{BASE_URL}/predict", json=payload, timeout=10) as response:
            if response.status == 200:
                data = await response.json()
                print(f"{model_name}: Predictions received: {data['predictions'][:5]}...")
            else:
                try:
                    error_data = await response.json()
                    error_detail = error_data.get('detail', 'No detail provided')
                except (aiohttp.ContentTypeError, ValueError):
                    error_detail = f"HTTP {response.status}"
 
                if response.status == 404:
                    print(f"Error for {model_name}: Model not found: {error_detail}")
                elif response.status == 429:
                    print(f"Error for {model_name}: Too many requests: {error_detail}")
                elif response.status == 500:
                    print(f"Error for {model_name}: Server error: {error_detail}")
                else:
                    print(f"Error for {model_name}: {error_detail}")
    except aiohttp.ClientError as e:
        print(f"Network error for {model_name}: {str(e)}")

async def async_predictions():
    async with aiohttp.ClientSession() as session:
        tasks = [
            predict_async(session, model["name"])
            for model in models
        ]
        await asyncio.gather(*tasks)

# Асинхронные предсказания
print("Starting asynchronous predictions...")
await async_predictions()

Starting asynchronous predictions...
logreg_model: Predictions received: [1, 1, 2, 1, 1]...
Error for gb_model: Model not found: Model not found: Model 'gb_model' not found on disk


## 4. Демонстрация остальных функций сервера

Поделаем разные операции с моделями: загрузку, выгрузку, удаление одной модели и удаление всех моделей. Используем синхронные вызовы через `requests` для простоты.

In [5]:
def manage_model(endpoint: str, model_name: str = None):
    """Универсальная функция для вызова управляющих эндпоинтов."""
    url = f"{BASE_URL}/{endpoint}"
    payload = {"config": {"model_name": model_name}} if model_name else {}
    try:
        response = requests.post(url, json=payload, timeout=10)
        response.raise_for_status()
        print(f"{endpoint}: {response.json()['message']}")
    except requests.exceptions.HTTPError as e:
        if response.status_code == 404:
            print(f"Error for {endpoint}: Model not found: {response.json()['detail']}")
        elif response.status_code == 500:
            print(f"Error for {endpoint}: Server error: {response.json()['detail']}")
        else:
            print(f"Error for {endpoint}: {str(e)}")
    except requests.exceptions.RequestException as e:
        print(f"Network error for {endpoint}: {str(e)}")

# Загрузка модели
manage_model("load", "logreg_model")

# Выгрузка модели
manage_model("unload", "logreg_model")

# Удаление модели
manage_model("remove", "logreg_model")

# Удаление всех моделей
manage_model("remove_all")

load: Model 'logreg_model' loaded successfully.
unload: Model 'logreg_model' unloaded successfully.
remove: Model 'logreg_model' removed successfully.
remove_all: All models removed successfully.


## 5. Проверка статуса сервера

In [6]:
try:
    response = requests.get(f"{BASE_URL}/status", timeout=10)
    response.raise_for_status()
    print("Server status:", json.dumps(response.json(), indent=2))
except requests.exceptions.RequestException as e:
    print(f"Network error: {str(e)}")

Server status: {
  "cpu_cores": 8,
  "max_processes": 7,
  "active_processes": 2,
  "active_requests": 0,
  "max_requests": 16,
  "model_dir": "/app/server/storage",
  "max_models": 10
}
